In [2]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from joblib import Parallel, delayed

# Load data
data = pd.read_csv('data.csv')

# Constants
num_train = 5_000  # Number of training samples
num_test = 50_000  # Number of test samples
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

# Function to compute loss for a single observation
def compute_loss(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )
    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = (
                data.loc[ind, 'm'].values
                + data.loc[ind, 'd'].values
                * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)
        )
    return np.sum(loss)

# Generate Training Data for GPR
print("Generating training data for GPR surrogate model...")
train_factors = np.random.normal(0, 1, (num_train, len(r) + len(data)))
train_losses = Parallel(n_jobs=-1)(delayed(compute_loss)(obs) for obs in train_factors)
train_factors = train_factors[:, :len(r)]  # Use factors only as inputs

# Train GPR Model
print("Training Gaussian Process Regression model...")
kernel = C(1.0, (1e-2, 1e2)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5)
gpr.fit(train_factors, train_losses)

# Predict Losses on Test Data
print("Predicting losses with GPR surrogate model...")
test_factors = np.random.normal(0, 1, (num_test, len(r)))
predicted_losses, sigma = gpr.predict(test_factors, return_std=True)

# Compute VaR and Variance for GPR
VaR_gpr = np.percentile(-predicted_losses, 99.9)
gpr_variance = np.var(predicted_losses)

print(f"GPR Surrogate Model VaR (99.9%): {VaR_gpr:.2f}")
print(f"GPR Surrogate Model Variance: {gpr_variance:.2f}")

KeyboardInterrupt: 